# Project Example

In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split



# EBM Model
# I cant get new packages imported for some reason so this still needs doing

from interpret.glassbox import ExplainableBoostingClassifier
from interpret import show

# NN Model
from sklearn.neural_network import MLPClassifier

from sklearn.metrics import mean_squared_error

from sklearn.inspection import PartialDependenceDisplay
#import PDF

np.random.seed(42)


hello


## Preprocessing Data
### Loading data

In [4]:
df = pd.read_csv('res/data/nasa.csv')
# Drop columns with irrelevant information, repetitions (such as KM and Miles) and static values (such as 'Orbiting Body' and 'Equinox')
df = df.drop(['Neo Reference ID', 'Name', 'Orbit ID', 'Close Approach Date',
                        'Epoch Date Close Approach', 'Orbit Determination Date', 'Est Dia in KM(max)', 'Est Dia in M(min)', 'Est Dia in M(max)', 'Est Dia in Miles(min)'
             ,'Est Dia in Miles(max)', 'Est Dia in Feet(min)', 'Est Dia in Feet(max)', 
             'Relative Velocity km per hr', 'Miles per hour', 'Miss Dist.(lunar)', 
             'Miss Dist.(kilometers)', 'Miss Dist.(miles)', 'Orbiting Body', 'Equinox'] , axis = 1)
#Convert target values to binary
df['Hazardous'] = df['Hazardous'].astype(int)
#df['Hazardous'].value_counts()



In [5]:
df.describe()

,Absolute Magnitude,Est Dia in KM(min),Relative Velocity km per sec,Miss Dist.(Astronomical),Orbit Uncertainity,Minimum Orbit Intersection,Jupiter Tisserand Invariant,Epoch Osculation,Eccentricity,Semi Major Axis,Inclination,Asc Node Longitude,Orbital Period,Perihelion Distance,Perihelion Arg,Aphelion Dist,Perihelion Time,Mean Anomaly,Mean Motion,Hazardous
count,4687.000000,4687.000000,4687.000000,4687.000000,4687.000000,4687.000000,4687.000000,4.687000e+03,4687.000000,4687.000000,4687.000000,4687.000000,4687.000000,4687.000000,4687.000000,4687.000000,4.687000e+03,4687.000000,4687.000000,4687.000000
mean,22.267865,0.204604,13.970811,0.256778,3.516962,0.082320,5.056111,2.457724e+06,0.382569,1.400264,13.373844,172.157275,635.582076,0.813383,183.932151,1.987144,2.457728e+06,181.167927,0.738242,0.161084
std,2.890972,0.369573,7.293223,0.145798,3.078307,0.090300,1.237818,9.202975e+02,0.180444,0.524154,10.936227,103.276777,370.954727,0.242059,103.513035,0.951519,9.442264e+02,107.501623,0.342627,0.367647
min,11.160000,0.001011,0.335504,0.000178,0.000000,0.000002,2.196000,2.450164e+06,0.007522,0.615920,0.014513,0.001941,176.557161,0.080744,0.006918,0.803765,2.450100e+06,0.003191,0.086285,0.000000
25%,20.100000,0.033462,8.432865,0.133420,0.000000,0.014585,4.049500,2.458000e+06,0.240858,1.000635,4.962341,83.081208,365.605031,0.630834,95.625916,1.266059,2.457815e+06,87.006918,0.453289,0.000000
50%,21.900000,0.110804,12.917889,0.265029,3.000000,0.047365,5.071000,2.458000e+06,0.372450,1.240981,10.311836,172.625393,504.947292,0.833153,189.761641,1.618195,2.457973e+06,185.718889,0.712946,0.000000
75%,24.500000,0.253837,18.077649,0.384154,6.000000,0.123593,6.019000,2.458000e+06,0.512411,1.678364,19.511681,255.026909,794.195972,0.997227,271.777557,2.451171,2.458108e+06,276.531946,0.984669,0.000000
max,32.100000,15.579552,44.633747,0.499884,9.000000,0.477891,9.025000,2.458020e+06,0.960261,5.072008,75.406667,359.905890,4172.231343,1.299832,359.993098,8.983852,2.458839e+06,359.917991,2.039000,1.000000


### Train Test split and Feature Normalization

In [6]:
# Split features from labels
X = df.drop(columns=['Hazardous']) #Features


intercept = np.ones(X.shape[0])
X['intercept'] = intercept

y = df['Hazardous'] #Target

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Get mean and standard deviation from train data
X_means = X_train.mean()
X_means['intercept'] = 0
X_std = X_train.std()
X_std['intercept'] = 1

# Normalize train and test data (Centering, unit variance)
X_train = X_train.apply(lambda x: (x-X_means)/X_std, axis = 1)
X_test = X_test.apply(lambda x: (x-X_means)/X_std, axis = 1)

In [7]:
X_train.head()
#X_train.describe()
#X_test.head()
#X_test.describe()

,Absolute Magnitude,Est Dia in KM(min),Relative Velocity km per sec,Miss Dist.(Astronomical),Orbit Uncertainity,Minimum Orbit Intersection,Jupiter Tisserand Invariant,Epoch Osculation,Eccentricity,Semi Major Axis,Inclination,Asc Node Longitude,Orbital Period,Perihelion Distance,Perihelion Arg,Aphelion Dist,Perihelion Time,Mean Anomaly,Mean Motion,intercept
1550,-0.689266,0.072314,-0.598330,0.995383,-1.145911,0.697868,0.539528,0.303691,-0.359068,-0.626681,-0.455599,0.340185,-0.616822,-0.347411,-0.115945,-0.601365,0.430256,0.584054,0.455593,1.0
1534,-0.270626,-0.179085,0.218480,-1.481291,-0.820943,-0.683117,-0.866822,0.303691,1.213790,0.549676,-1.072771,0.596696,0.444395,-0.573302,0.381695,0.747852,0.089043,-0.833465,-0.852121,1.0
2924,0.706202,-0.425979,-0.916298,-1.532630,1.128870,-0.709440,-1.276224,0.303691,0.809159,1.468044,-1.151647,-1.656467,1.425311,0.893435,1.499150,1.388759,0.051181,-0.989201,-1.271773,1.0
2697,0.392222,-0.377783,-0.950869,-1.363476,0.803902,-0.408552,-0.909315,0.303691,0.366359,0.760938,-0.984742,1.222106,0.659064,0.754647,-0.753319,0.646664,0.044714,-0.746645,-0.973754,1.0
1006,0.043355,-0.294701,-0.116918,1.480220,-0.820943,-0.685082,0.657200,0.303691,1.081660,-0.735058,-0.989824,-0.518602,-0.702008,-1.604946,0.751790,-0.404806,0.460097,0.234726,0.680808,1.0


### Getting feature Importances

In [8]:
# Random Forest
rf = RandomForestClassifier()
rf.fit(X_train, y_train)

# Sort by feature importance
importances = rf.feature_importances_
feature_names = X.columns
feature_importance_df = pd.DataFrame({'Feature': feature_names, 'Importance': importances})
feature_importance_df = feature_importance_df.sort_values(by='Importance', ascending=False)
print(feature_importance_df)

                         Feature  Importance
5     Minimum Orbit Intersection    0.394994
1             Est Dia in KM(min)    0.188988
0             Absolute Magnitude    0.147273
4             Orbit Uncertainity    0.055652
10                   Inclination    0.041311
13           Perihelion Distance    0.037068
3       Miss Dist.(Astronomical)    0.022112
8                   Eccentricity    0.018174
2   Relative Velocity km per sec    0.014310
15                 Aphelion Dist    0.011528
14                Perihelion Arg    0.011224
6    Jupiter Tisserand Invariant    0.009885
18                   Mean Motion    0.008845
9                Semi Major Axis    0.008375
16               Perihelion Time    0.008336
17                  Mean Anomaly    0.007512
12                Orbital Period    0.006661
11            Asc Node Longitude    0.006283
7               Epoch Osculation    0.001467
19                     intercept    0.000000


In [9]:
# TODO: Check if these feature importances still are important
print(df.columns)

#Select the most important features
important_features = [
    'Minimum Orbit Intersection',
    'Absolute Magnitude',
    'Est Dia in KM(min)',
    'Orbit Uncertainity',
    'Inclination', 
    'Eccentricity',
    'Miss Dist.(Astronomical)',
    'Relative Velocity km per sec',
    'Aphelion Dist',
    'Perihelion Arg',
    'Hazardous',
]

df_selected = df[important_features]
df_selected.head()

Index(['Absolute Magnitude', 'Est Dia in KM(min)',
       'Relative Velocity km per sec', 'Miss Dist.(Astronomical)',
       'Orbit Uncertainity', 'Minimum Orbit Intersection',
       'Jupiter Tisserand Invariant', 'Epoch Osculation', 'Eccentricity',
       'Semi Major Axis', 'Inclination', 'Asc Node Longitude',
       'Orbital Period', 'Perihelion Distance', 'Perihelion Arg',
       'Aphelion Dist', 'Perihelion Time', 'Mean Anomaly', 'Mean Motion',
       'Hazardous'],
      dtype='object')


,Minimum Orbit Intersection,Absolute Magnitude,Est Dia in KM(min),Orbit Uncertainity,Inclination,Eccentricity,Miss Dist.(Astronomical),Relative Velocity km per sec,Aphelion Dist,Perihelion Arg,Hazardous
0,0.025282,21.6,0.127220,5,6.025981,0.425549,0.419483,6.115834,2.005764,57.257470,1
1,0.186935,21.3,0.146068,3,28.412996,0.351674,0.383014,18.113985,1.497352,313.091975,0
2,0.043058,20.3,0.231502,0,4.237961,0.348248,0.050956,7.590711,1.966857,248.415038,1
3,0.005512,27.4,0.008801,6,7.905894,0.216578,0.285322,11.173874,1.527904,18.707701,0
4,0.034798,21.6,0.127220,1,16.793382,0.210448,0.407832,9.840831,1.483543,158.263596,1


### Data variations

In [10]:
X_train_sets = {}
X_test_sets = {}

conditions = ['baseline', 'normal']

#Baseline
X_train['feat'] = np.zeros(X_train.shape[0])
X_test['feat'] = np.zeros(X_test.shape[0])
X_train_sets[conditions[0]] = X_train.copy(deep = True)
X_test_sets[conditions[0]] = X_test.copy(deep = True)

# Normal Distribution
X_train['feat'] = np.random.normal(loc = 0, scale = 1, size = X_train.shape[0])
X_test['feat'] = np.random.normal(loc = 0, scale = 1, size = X_test.shape[0])
X_train_sets[conditions[1]] = X_train.copy(deep = True)
X_test_sets[conditions[1]] = X_test.copy(deep = True)

# MORE: what conditions are worth checking out?



In [11]:
print(X_train_sets['baseline']['feat'].head())

1550    0.0
1534    0.0
2924    0.0
2697    0.0
1006    0.0
Name: feat, dtype: float64


## Models
### NN Parameter cross-validation

In [ ]:
hidden_layers = (100, 3) # 3 layers, 100 neurons
alphas = np.logspace(-1, 1, 5)

CV_RMSE = []

NN_classifiers = []
for i in range(len(alphas)):
    alpha = alphas[i]
    NN_classifiers.append(
        MLPClassifier(
            hidden_layer_sizes=hidden_layers,
            activation='relu', # relu is required
            solver='adam', # can change if u like
            alpha = alpha
            )
    )
    
    #
    # NN_classifiers[i].fit(cv_xtrain, cv_ytrain)
    # preds = NN_classifiers[i]
    #WIP
    pass

### Setup

In [12]:
models_EBM = {}
models_NN = {}

In [13]:
#alpha = ? # Obtain through CV above

for cond in conditions:
    models_NN['baseline'] = MLPClassifier(
        hidden_layer_sizes=100,
        activation='lbfgs',
        alpha = 0.1, # Strength of L2 regularization. Needs finetuning first?
        random_state=1,
        solver='adam'
        # MORE FINETUNING REQUIRED
    )

    # Basic params
    models_EBM['baseline'] = ExplainableBoostingClassifier()  
    

### Fitting

In [ ]:
for condition in conditions:
    models_NN[condition].fit(X = X_train_sets[condition], 
                             y = y_train)
    
    models_EBM[condition].fit(X = X_train_sets[condition],
                              y = y_train)

### Prediction

In [ ]:
predictions_NN = {}

### Model Performance

## Feature Importances
